# Preprocessamento
### Faça as tarefas usuais de processameno de textos:
    Conversão de caracteres maiúsculos para minúsculos
    Remoção de pontuação
    Remoção de stop words
    Steming dos termos
    Remoção dos termos que aparecem em um só documento ou que aparecem em todos.

### Converta os textos processados em um bag of words no formato binário e no formato de term frequency.

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/tales/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
import nltk, sklearn, string, re, numpy, scipy, math
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
from nltk.stem.snowball import EnglishStemmer
from datetime import datetime, timedelta
from scipy.sparse import csr_matrix

data = load_files('./ex5-files/filesk/', encoding='utf8', shuffle=True, 
                                                          load_content=False)

def stemmingTokens(text):
    text = re.sub(r'[^\w\s]','',text) # Remoção de pontuação
    tokens = nltk.word_tokenize(text)
    stemmer = EnglishStemmer()
    stems = [stemmer.stem(i) for i in tokens] # Stemming dos termos
    return stems

qntDocs = 5000
vectorizer = CountVectorizer(
        input='filename', 
        binary=False,
        strip_accents='unicode',
        lowercase=True,            # Conversão para minúsculas
        tokenizer=stemmingTokens,  # Remoção de pontuação e stemming dos termos
        stop_words='english',      # Remoção de stop words
        min_df=2,                  # Remoção dos termos em um só documento...
        max_df=qntDocs-1)          # ... ou que aparecem em todos 

countSM = vectorizer.fit_transform(data['filenames'][:qntDocs])
binSM = countSM.sign()
y = data['target'][:qntDocs]

/home/tales/virtualenvs/ML/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'forti', 'henc', 'hereaft', 'herebi', 'howev', 'hundr', 'inde', 'mani', 'meanwhil', 'moreov', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'togeth', 'twelv', 'twenti', 'veri', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


### Divida o conjunto em  4000 documentos de treino e 1000 de teste aleatoriamente.

In [2]:
x_train, x_test, y_train, y_test = train_test_split(countSM, y, test_size=0.2, 
                                                                      random_state=0)
x_trainBin, x_testBin, y_trainBin, y_testBin = train_test_split(binSM, y, 
                                                       test_size=0.2, random_state=0)

# Parte 2 - Naive Bayes
### Rode o *Naive Bayes* na matriz binaria.

In [4]:
binScore = BernoulliNB(binarize=None).fit(x_trainBin, y_trainBin).score(x_testBin, 
                                                                        y_testBin)
print("Acurácia média usando BernoulliNB na matriz binária: {0:.3f}".format(binScore))

Acurácia média usando BernoulliNB na matriz binária: 0.744


A classe BernoulliNB poderia receber a matriz de frequência e criar a matriz binária internamente mas esta abordagem mostrou resultados inferiores.

### Rode o *Naive Bayes*  na matriz de term-frequency.

In [5]:
freqScore = MultinomialNB().fit(x_train, y_train).score(x_test, y_test)
print("Acurácia média usando MultinomialNB na matriz de frequência de termos: {0:.3f}"
                                                                   .format(freqScore))

Acurácia média usando MultinomialNB na matriz de frequência de termos: 0.798


Como pode ser avaliado, temos cerca de 5.4% a mais de acurácia usando a matriz de term frequency no lugar da matriz binária.

# PCA e outros classificadores
### Rode o PCA na matriz de term-frequency mantendo 99% da variância original com o TruncatedSVD

In [6]:
n_words = int(3000/2)
totalVariance = 0
while (totalVariance < 0.99):
    n_words *= 2
    fullSVD = TruncatedSVD(n_components=n_words, n_iter=1)
    fullSVD.fit(countSM, y)

    n_words = 0
    totalVariance = 0
    for variance in sorted(fullSVD.explained_variance_ratio_, reverse=True):
        totalVariance += variance
        n_words += 1
        print(totalVariance, end='\r')
        if (totalVariance >= 0.99):
            break
    print("Com {0} palavras/componentes há variância igual à {1:.2f}%"
              .format(n_words, totalVariance*100))

SVD = TruncatedSVD(n_components=n_words, n_iter=5)
x_PCA = SVD.fit_transform(countSM, y)
x_trainPCA, x_testPCA, y_trainPCA, y_testPCA = train_test_split(x_PCA, y, 
                                                        test_size=0.2, random_state=0)


Com 2851 palavras/componentes há variância igual à 99.00%


### Rode os seguintes algoritmos na matriz com o número de dimensões reduzidas, buscando hiperparâmetros.

#### SVM com RBF (modo one vs all)

In [17]:
# Conjunto de hiperparâmetros testados para otimização do SVM
Cs = [2**-5, 2**1, 2**5, 2**10]
gammas = [2**-15, 2**-10, 2**0, 2**5]

print("Teste do SVM")

# Procura o melhor C e gamma com 3 folds (kernel='rbf' por padrão)
gscv = GridSearchCV(
            OneVsRestClassifier(SVC()), 
            {'estimator__C':Cs, 'estimator__gamma':gammas},
            n_jobs=3
       ).fit(x_trainPCA, y_trainPCA)

C = int(math.log(gscv.best_params_["estimator__C"], 2))
gamma = int(math.log(gscv.best_params_["estimator__gamma"], 2))
svmScore = gscv.score(x_testPCA, y_testPCA)
print('C = 2^{:d}, gamma = 2^{:d}; Acurácia = {:.12f}'.format(
            C, 
            gamma, 
            svmScore))

Teste do SVM
C = 2^1, gamma = 2^-10; Acurácia = 0.837000000000


#### Gradient boosting

In [8]:
# Conjunto de hiperparâmetros testados para otimização do GBM
n_estimators = [30, 70, 100]
learning_rates = [0.1, 0.05]
max_depth = [5]

print("Teste do GBM")

# Procura os melhores hiperparâmetros 3 folds (StratifiedKFold)
gscv = GridSearchCV(
            GradientBoostingClassifier(), {
                'n_estimators':n_estimators, 
                'learning_rate':learning_rates, 
                'max_depth':max_depth},
            n_jobs=3
       ).fit(x_trainPCA, y_trainPCA)

gbmScore = gscv.score(x_testPCA, y_testPCA)
print('n_estimators = {:3d}, learning_rate = {:.2f}; Acurácia = {:.12f}'.format(
            gscv.best_params_["n_estimators"], 
            gscv.best_params_["learning_rate"], 
            gbmScore))

Teste do GBM
n_estimators = 100, learning_rate = 0.10; Acurácia = 0.810000000000


#### Random Forest 

In [10]:
# Conjunto de hiperparâmetros testados para otimização do RF
max_features = [3, 5, 7]
n_estimators = [50, 100, 500, 1000]

print("Teste do RF")

# Procura os melhores hiperparâmetros 3 folds (StratifiedKFold)
gscv = GridSearchCV(
            RandomForestClassifier(), 
            {'n_estimators':n_estimators, 'max_features':max_features},
            n_jobs=3
       ).fit(x_trainPCA, y_trainPCA)

# Guarda a acurácia da Random Forest para os hiperparâmetros do fold atual
rfScore = gscv.score(x_testPCA, y_testPCA)
print('n_estimators = {:d}, max_features = {:d}; Acurácia = {:.12f}'.format(
            gscv.best_params_["n_estimators"], 
            gscv.best_params_["max_features"],
            rfScore))

Teste do RF
n_estimators = 50, max_features = 7; Acurácia = 0.564000000000


# Considerações Finais
### Qual o melhor classificador dos testados?
Como podemos ver, o SVC com RBF tem a melhor acurácia: 0.837, mas exige um pesado processamento. Abaixo, um treinamento com toda a base.

In [18]:
gscv = GridSearchCV(
            SVC(), 
            {'C':Cs, 'gamma':gammas},
            n_jobs=3
       ).fit(x_PCA, y)
print(gscv.best_estimator_)

SVC(C=32, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.0009765625, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


Vale reforçar que talvez o ganho de acurácia não compense o tempo de processamento exigido pelo SVM. Os métodos Naive tem acurácia satisfatória e exigem muito menos.